In [1]:
%pip install rasterio
%pip install scikit-learn
import pandas as pd
import numpy as np
%pip install matplotlib
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import rasterio
import numpy as np
import pandas as pd

# Load the GeoTIFF
file_path = r"D:\no2-prediction-project\NO2_mean_2019_06_01_to_2024_11_09_1.tif"
with rasterio.open(file_path) as src:
    no2_data = src.read(1)  # Read the first band (assuming NO2 data is in the first band)
    transform = src.transform  # Affine transform for the pixel coordinates
    width = src.width
    height = src.height

# Generate coordinates for each pixel (latitude and longitude)
latitudes = []
longitudes = []

# Loop over the pixels to extract the latitudes and longitudes
for row in range(height):
    for col in range(width):
        # Convert pixel coordinates to lat/lon (using the affine transform)
        lon, lat = transform * (col, row)  # Convert (col, row) to (lon, lat)
        latitudes.append(lat)
        longitudes.append(lon)

# Flatten NO2 data to match the latitudes and longitudes
no2_values = no2_data.flatten()

# Create a DataFrame for machine learning (latitude, longitude, NO2 level)
data = pd.DataFrame({
    'latitude': latitudes,
    'longitude': longitudes,
    'NO2_level': no2_values
})

# Convert data types for efficiency
data['latitude'] = data['latitude'].astype('float32')
data['longitude'] = data['longitude'].astype('float32')
data['NO2_level'] = data['NO2_level'].astype('float32')

# Optionally, drop rows with missing values (if any)
data.dropna(subset=['NO2_level'], inplace=True)


In [12]:
import pandas as pd

# Flatten the NO₂ data and create a DataFrame
no2_flat = no2_data.flatten()

# Generate row and column indices to represent spatial positions
rows, cols = np.indices(no2_data.shape)
latitude = rows.flatten()
longitude = cols.flatten()

# Create a DataFrame with NO₂ density, latitude, and longitude as features
data_df = pd.DataFrame({
    'NO2_density': no2_flat,
    'latitude': latitude,
    'longitude': longitude
})

# Remove any missing values (if any)
data_df = data_df.dropna()
data_df.head()


,NO2_density,latitude,longitude
0,0.000057,0,0
1,0.000057,0,1
2,0.000057,0,2
3,0.000057,0,3
4,0.000057,0,4


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Separate features (latitude, longitude) and target (NO₂ density)
X = data_df[['latitude', 'longitude']]
y = data_df['NO2_density']

# Normalize features
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)


In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib

# Sample a subset of the data (10% for testing purposes)
data_sampled = data.sample(frac=0.1, random_state=42)  # 10% of the data

# Define features (X) and target (y)
X = data_sampled[['latitude', 'longitude']]  # Example features (latitude and longitude)
y = data_sampled['NO2_level']  # Target variable (NO2 levels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest model with fewer trees to reduce memory usage
model = RandomForestRegressor(n_estimators=10, random_state=42)  # Reduced number of trees

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

# Save the trained model
joblib.dump(model, "random_forest_no2_model.pkl")


Mean Squared Error: 3.2457554079298145e-13
Mean Absolute Error: 2.1715329100446446e-07


['random_forest_no2_model.pkl']